# Lista 4
### NCG-040 - Introdução à Visão Computacional

**Instruções:**

 1. Comentários nos códigos desenvolvidos.
 1. Caso exista uma pergunta, deve ser respondida como comentários no arquivo ou em uma nova célula de texto.
 1. Digitar seu nome e DRE abaixo.
 1. Ao finalizarem a lista, renomeie ou salve uma cópia do arquivo no formato NomeSobrenome_ListaX.ipynb
 1. Enviar via Dropbox até 25/06/2025 - https://www.dropbox.com/request/Duz3Wq2nsdIYQVp7PVvH

`Nome: `

`DRE: `

## Opção 1: YOLO (You Only Look Once)

1. **Configuração do Ambiente**
   - Instale as dependências necessárias (e.g., `torch`, `opencv-python`, `yolov5`).
   - Importe as bibliotecas.

2. **Inferência com Modelo Pré-Treinado**
   - Carregue o modelo YOLO de sua preferência.
   - Aplique a detecção em imagens de demonstração.
   - Exiba resultados com bounding boxes.

3. **Treinamento em Dataset Personalizado**
   - Prepare um pequeno dataset de imagens anotadas (`images/` e `labels/`).
   - Ajuste o arquivo de configuração para apontar para seu dataset.
   - Realize treinamento rápido (apenas algumas épocas) e monitore métricas.

4. **Avaliação e Métricas**
   - Calcule Precision, Recall e mAP para seu modelo.
   - Plote gráficos de Loss e mAP durante o treinamento.

5. **Melhorias e Experimentação**
   - Ajuste hiperparâmetros (tamanho de batch, learning rate) e compare resultados.
   - Teste diferentes tamanhos de input (320x320 vs 640x640).


In [ ]:
# Parte 1: YOLO

# 1. Instalação de dependências (execute apenas no Colab)
!pip install torch torchvision opencv-python yolov5

# 2. Importações
import torch
import cv2
from yolov5 import YOLOv5  # ou use import ultralytics

# 3. Carregar modelo pré-treinado
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# 4. Inferência em imagem de exemplo
img_path = 'caminho/para/sua/imagem.jpg'
img = cv2.imread(img_path)
results = model(img)

# 5. Mostrar resultados
results.render()  # desenha bounding boxes
from matplotlib import pyplot as plt
plt.imshow(cv2.cvtColor(results.imgs[0], cv2.COLOR_BGR2RGB))
plt.axis('off')


## Opção 2: Vision Transformers (ViT)

1. **Introdução ao ViT**
   - Entenda a arquitetura de patch embedding e self-attention.

2. **Implementação de Patch Embedding**
   - Escreva sua própria classe `PatchEmbedding` em PyTorch.

3. **Uso de Modelo Pré-Treinado**
   - Utilize um ViT pré-treinado (`timm`, `torchvision.models`) para classificação.
   - Carregue e avalie em um conjunto de validação leve (e.g., CIFAR-10).

4. **Visualização de Mapas de Atenção**
   - Extraia os pesos de atenção de uma camada específica.
   - Plote um mapa de atenção sobre uma imagem de entrada.

5. **Fine-Tuning**
   - Congele as camadas iniciais e treine apenas o cabeçalho de classificação num pequeno dataset.
   - Compare acurácia antes e depois do fine-tuning.


In [ ]:
# Parte 2: Vision Transformers

# 1. Instalação de dependências
!pip install torch torchvision timm

# 2. Importações
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import timm

# 3. Implementação de PatchEmbedding
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=32, patch_size=4, in_channels=3, embed_dim=64):
        super().__init__()
        self.patch_size = patch_size
        self.projection = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
    def forward(self, x):
        x = self.projection(x)
        x = x.flatten(2).transpose(1, 2)
        return x

# 4. Carregar dataset e modelo pré-treinado
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])
val_dataset = CIFAR10(root='.', train=False, download=True, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

model_vit = timm.create_model('vit_tiny_patch16_224', pretrained=True)
model_vit.head = nn.Linear(model_vit.head.in_features, 10)  # ajuste para CIFAR-10

# 5. Avaliação inicial
model_vit.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = model_vit(images)
        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
print(f'Acurácia inicial: {correct/total:.2%}')
